In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./3.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"extras","groupname":"' + input_.loc[i, 'Category'] + '","subgroupname":"' + input_.loc[i, 'Subcategory'] + '","parttype":"' + input_.loc[i, 'Type_Code'] + '","nodetype":"parttype"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')

                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                    
                # = = = = = = = = = = = = = = =

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]/tr/td/input[contains(@id, "listing_data_supplemental")]/@value')

                # = = = = = = = = = = = = = = =
                
                list_tag = [json.loads(part)['sortgrouptext'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [json.loads(part)['catalogname'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_part_number = [json.loads(part)['partnumber'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                        'No.': int(input_.loc[i, 'No.']),
                                        'Make': '',
                                        'Year': '',
                                        'Model': '',
                                        'Engine': '',
                                        'Car_Code': '',
                                        'Category': input_.loc[i, 'Category'],
                                        'Subcategory': input_.loc[i, 'Subcategory'],
                                        'Type': input_.loc[i, 'Type'],
                                        'Type_Code': input_.loc[i, 'Type_Code'],
                                        'Tab_Url': input_.loc[i, 'Url'],
                                        'Part_Index': [(index+1) for index in range(len(list_part))],
                                        'Tag': list_tag,
                                        'Manufacturer': list_manufacturer,
                                        'Part_Number': list_part_number,
                                        'Vehcile': '',
                                        'OE': '',
                                        'Note_1': '',
                                        'Note_2': '',
                                        'Pic': '',
                                        'Url': '',
                                        'Info': '',
                                        'Src': '',
                                        'Part_Type': '',
                                        'Part_Key': '',
                                        'JOIN_MPN': [manufacturer+';'+part_number for manufacturer, part_number in zip(list_manufacturer, list_part_number)]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Category': input_.loc[i, 'Category'],
                                     'Subcategory': input_.loc[i, 'Subcategory'],
                                     'Type': input_.loc[i, 'Type'],
                                     'Type_Code': input_.loc[i, 'Type_Code'],
                                     'Url': input_.loc[i, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Category'] + ' - ' + input_.loc[i, 'Subcategory'] + ' - ' + input_.loc[i, 'Type'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Subcategory', 'Type', 'Part_Index'], ascending=[True, True, True, True])
output_correct.to_excel('./4.part_number-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Subcategory', 'Type'], ascending=[True, True, True])
    output_error.to_excel('./part_number_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：1763

Apparel & Gifts - RockAuto Gift - Lanyard  <->  [ok] - 剩余数量：1743 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 1/32"      (26.2mm) Belt  <->  [ok] - 剩余数量：1742 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 5/32"      (29.3mm) Belt  <->  [ok] - 剩余数量：1741 - 当前时间：18:24:50
Apparel & Gifts - RockAuto Gift - Magnets  <->  [ok] - 剩余数量：1740 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 15/32"      (37.3mm) Belt  <->  [ok] - 剩余数量：1739 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 11/32"      (34.1mm) Belt  <->  [ok] - 剩余数量：1738 - 当前时间：18:24:50
Apparel & Gifts - RockAuto Gift - Gift Certificate  <->  [ok] - 剩余数量：1737 - 当前时间：18:24:50
Apparel & Gifts - RockAuto Apparel - Hat  <->  [ok] - 剩余数量：1736 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 3/8"      (34.9mm) Belt  <->  [ok] - 剩余数量：1735 - 当前时间：18:24:50
Apparel & Gifts - RockAuto Apparel - Hoodie  <->  [ok] - 剩余数量：1734 - 当前时间：18:24:50
Belt Drive - ATV / UTV Belt - 1 1/8"      (28.6mm) Belt  <->  [ok] - 剩余数量：1733 - 当前时间：18:24:50

Body & Lamp Assembly - Accessory Lighting - Auxiliary Light  <->  [ok] - 剩余数量：1661 - 当前时间：18:25:09
Body & Lamp Assembly - Accessory Lighting - Identification Light Bar  <->  [ok] - 剩余数量：1660 - 当前时间：18:25:09
Body & Lamp Assembly - Accessory Lighting - Flood / Spot Light  <->  [ok] - 剩余数量：1659 - 当前时间：18:25:09
Belt Drive - Refrigeration V-Belt - Double-Sided Belt  <->  [ok] - 剩余数量：1658 - 当前时间：18:25:09
Body & Lamp Assembly - Accessory Lighting - LED Warning Light  <->  [ok] - 剩余数量：1657 - 当前时间：18:25:09
Body & Lamp Assembly - Accessory Lighting - Docking Light  <->  [ok] - 剩余数量：1656 - 当前时间：18:25:10
Body & Lamp Assembly - Accessory Lighting - Off-Road Light Bar Damper  <->  [ok] - 剩余数量：1655 - 当前时间：18:25:10
Body & Lamp Assembly - Accessory Lighting - Oval Strobe Light  <->  [ok] - 剩余数量：1654 - 当前时间：18:25:10
Body & Lamp Assembly - Accessory Lighting - Round Strobe Light  <->  [ok] - 剩余数量：1653 - 当前时间：18:25:11
Body & Lamp Assembly - Accessory Lighting - Mini Utility Light  <->  [ok] - 剩余数量：1652 - 

Body & Lamp Assembly - Clearance / Side Marker Lamp - Rectangular Clearance & Side Marker Lamp  <->  [ok] - 剩余数量：1581 - 当前时间：18:25:28
Body & Lamp Assembly - Fluid / Chemical - Glass Primer  <->  [ok] - 剩余数量：1580 - 当前时间：18:25:29
Body & Lamp Assembly - Fluid / Chemical - Glass Adhesive  <->  [ok] - 剩余数量：1579 - 当前时间：18:25:29
Body & Lamp Assembly - Clearance / Side Marker Lamp - Mini Marker Lamp Lens  <->  [ok] - 剩余数量：1578 - 当前时间：18:25:29
Body & Lamp Assembly - Fender - Fender Bolt  <->  [ok] - 剩余数量：1577 - 当前时间：18:25:29
Body & Lamp Assembly - Cargo Management - Tailgate Lock  <->  [ok] - 剩余数量：1576 - 当前时间：18:25:29
Body & Lamp Assembly - Fluid / Chemical - Lens Cleaner  <->  [ok] - 剩余数量：1575 - 当前时间：18:25:30
Body & Lamp Assembly - Fluid / Chemical - Filler  <->  [ok] - 剩余数量：1574 - 当前时间：18:25:30
Body & Lamp Assembly - Clearance / Side Marker Lamp - Wedge Clearance & Side Marker Lamp  <->  [ok] - 剩余数量：1573 - 当前时间：18:25:30
Body & Lamp Assembly - Fluid / Chemical - Rubbing Compound  <->  [ok] - 剩

Body & Lamp Assembly - Painting - Paint Sprayer Adapter  <->  [ok] - 剩余数量：1498 - 当前时间：18:25:56
Body & Lamp Assembly - Lift Support - 6"-7"+ Extended Length Lift Support  <->  [ok] - 剩余数量：1497 - 当前时间：18:25:56
Body & Lamp Assembly - Parking / Turn Signal Lamp - Park & Turn Signal Lamp  <->  [ok] - 剩余数量：1496 - 当前时间：18:25:57
Body & Lamp Assembly - Painting - Paint Brush  <->  [ok] - 剩余数量：1495 - 当前时间：18:25:57
Body & Lamp Assembly - Parking / Turn Signal Lamp - Oval LED Turn Signal Lamp  <->  [ok] - 剩余数量：1494 - 当前时间：18:25:57
Body & Lamp Assembly - Parking / Turn Signal Lamp - Mid-Turn Lamp  <->  [ok] - 剩余数量：1493 - 当前时间：18:25:57
Body & Lamp Assembly - Parking / Turn Signal Lamp - ECE Compliant Turn Signal Lamp  <->  [ok] - 剩余数量：1492 - 当前时间：18:25:57
Body & Lamp Assembly - Lift Support - 19"+ Extended Length Lift Support  <->  [ok] - 剩余数量：1491 - 当前时间：18:25:58
Body & Lamp Assembly - Outside Mirror - Universal Mirror Assembly  <->  [ok] - 剩余数量：1490 - 当前时间：18:25:58
Body & Lamp Assembly - Painting 

Body & Lamp Assembly - Tool - Dent Puller  <->  [ok] - 剩余数量：1418 - 当前时间：18:26:21
Body & Lamp Assembly - Tool - Headlamp Adjuster  <->  [ok] - 剩余数量：1417 - 当前时间：18:26:22
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel Hitch Bracket  <->  [ok] - 剩余数量：1416 - 当前时间：18:26:22
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel Hitch Plate  <->  [ok] - 剩余数量：1415 - 当前时间：18:26:22
Body & Lamp Assembly - Tape - Seam Sealer Tape  <->  [ok] - 剩余数量：1414 - 当前时间：18:26:22
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel / Gooseneck Hitch Mount Kit  <->  [ok] - 剩余数量：1413 - 当前时间：18:26:23
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel Hitch Leg  <->  [ok] - 剩余数量：1412 - 当前时间：18:26:23
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel Hitch Rail  <->  [ok] - 剩余数量：1411 - 当前时间：18:26:23
Body & Lamp Assembly - Towing - Fifth Wheel - Fifth Wheel Safety Chain Anchor  <->  [ok] - 剩余数量：1410 - 当前时间：18:26:23
Body & Lamp Assembly - Towing - Fifth Wheel - Gooseneck Hitch Cover  

...

Hoses/Lines & Clamps - Exhaust & Emission Hose - Exhaust Hose  <->  [ok] - 剩余数量：399 - 当前时间：18:32:15
Hoses/Lines & Clamps - Fitting - Connector - Double Compression Connector  <->  [ok] - 剩余数量：398 - 当前时间：18:32:15
Hoses/Lines & Clamps - Fitting - Connector - Fuel Hose / Line Connector - 90º  <->  [ok] - 剩余数量：397 - 当前时间：18:32:15
Hoses/Lines & Clamps - Clamp - < 1" Clamp  <->  [ok] - 剩余数量：396 - 当前时间：18:32:16
Hoses/Lines & Clamps - Coolant Hose - Coolant Hose (Straight) - Silicone  <->  [ok] - 剩余数量：395 - 当前时间：18:32:16
Hoses/Lines & Clamps - Driveway Signal Device Hose - Driveway Signal Device Hose  <->  [ok] - 剩余数量：394 - 当前时间：18:32:17
Hoses/Lines & Clamps - Copper Tubing - Copper Tubing  <->  [ok] - 剩余数量：393 - 当前时间：18:32:17
Hoses/Lines & Clamps - Clamp - 2" - 3" Clamp  <->  [ok] - 剩余数量：392 - 当前时间：18:32:18
Hoses/Lines & Clamps - Fitting - Connector - Bubble Flare Connector  <->  [ok] - 剩余数量：391 - 当前时间：18:32:18
Hoses/Lines & Clamps - Fitting - Connector - Fuel Hose / Line Connector - Straight

Hoses/Lines & Clamps - Tool - Heater Hose Coupler Remover  <->  [ok] - 剩余数量：315 - 当前时间：18:32:41
Hoses/Lines & Clamps - Nylon Tubing Fitting - Nylon Tubing Insert  <->  [ok] - 剩余数量：314 - 当前时间：18:32:42
Hoses/Lines & Clamps - Tool - Hose Clamp Pliers  <->  [ok] - 剩余数量：313 - 当前时间：18:32:42
Hoses/Lines & Clamps - Tool - Hose / Tube Cutter  <->  [ok] - 剩余数量：312 - 当前时间：18:32:43
Hoses/Lines & Clamps - Tool - Disconnect Tool  <->  [ok] - 剩余数量：311 - 当前时间：18:32:43
Hoses/Lines & Clamps - Tool - Hose Removal Tool  <->  [ok] - 剩余数量：310 - 当前时间：18:32:43
Hoses/Lines & Clamps - Tool - Hydraulic Hose Plug  <->  [ok] - 剩余数量：309 - 当前时间：18:32:43
Hoses/Lines & Clamps - Tool - Hose Shaping Coil  <->  [ok] - 剩余数量：308 - 当前时间：18:32:43
Hoses/Lines & Clamps - Tool - Hose / Tube Bender  <->  [ok] - 剩余数量：307 - 当前时间：18:32:44
Hoses/Lines & Clamps - Tool - Hose Clamp Vise  <->  [ok] - 剩余数量：306 - 当前时间：18:32:44
Hoses/Lines & Clamps - Transmission Fluid Cooler Line / Hose - Transmission Fluid Cooler Line / Hose  <->  [ok] 

Interior - Gauge - Boost Gauge  <->  [ok] - 剩余数量：222 - 当前时间：18:33:13
Interior - Cup Holder - Cup Holder  <->  [ok] - 剩余数量：221 - 当前时间：18:33:13
Interior - Gauge - Engine Oil Temperature Gauge  <->  [ok] - 剩余数量：220 - 当前时间：18:33:14
Interior - Fluid / Chemical - Rear View Mirror Adhesive  <->  [ok] - 剩余数量：219 - 当前时间：18:33:14
Interior - Cigarette Lighter - Cigarette Lighter Receptacle  <->  [ok] - 剩余数量：218 - 当前时间：18:33:15
Interior - Clock - Clock  <->  [ok] - 剩余数量：217 - 当前时间：18:33:16
Interior - Gauge - Fuel Level Gauge  <->  [ok] - 剩余数量：216 - 当前时间：18:33:17
Interior - Gauge - Fuel Pressure Gauge  <->  [ok] - 剩余数量：215 - 当前时间：18:33:18
Interior - Gauge - Oil Pressure Gauge Fitting  <->  [ok] - 剩余数量：214 - 当前时间：18:33:18
Interior - Fluid / Chemical - Patch Repair  <->  [ok] - 剩余数量：213 - 当前时间：18:33:18
Fuel & Air - Carburetor - Carburetor Choke Cable  <->  [ok] - 剩余数量：212 - 当前时间：18:33:18
Interior - Floor Mat - Floor Mat  <->  [ok] - 剩余数量：211 - 当前时间：18:33:19
Interior - Gauge - Oil Pressure Gauge Insta

Steering - Steering Damper - Steering Damper  <->  [ok] - 剩余数量：122 - 当前时间：18:33:45
Steering - Tie Rod End - Tie Rod End Boot  <->  [ok] - 剩余数量：121 - 当前时间：18:33:46
Suspension - Leaf Spring - Leaf Spring 1000-1950 lb Load Rating  <->  [ok] - 剩余数量：120 - 当前时间：18:33:46
Suspension - Leaf Spring - 9/16" Bolt  <->  [ok] - 剩余数量：119 - 当前时间：18:33:46
Suspension - Leaf Spring - Leaf Spring Insulator  <->  [ok] - 剩余数量：118 - 当前时间：18:33:47
Suspension - Leaf Spring - Leaf Spring <1000 lb Load Rating  <->  [ok] - 剩余数量：117 - 当前时间：18:33:48
Suspension - Leaf Spring - 3/8" Bolt  <->  [ok] - 剩余数量：116 - 当前时间：18:33:48
Suspension - Shock Absorber - Shock Bushing  <->  [ok] - 剩余数量：115 - 当前时间：18:33:48
Suspension - Shock Absorber - Bump Stop  <->  [ok] - 剩余数量：114 - 当前时间：18:33:49
Suspension - Tool - Ball Joint Press  <->  [ok] - 剩余数量：113 - 当前时间：18:33:49
Suspension - Tool - Ball Joint Press Adapter  <->  [ok] - 剩余数量：112 - 当前时间：18:33:49
Suspension - Tool - Ball Joint Remover  <->  [ok] - 剩余数量：111 - 当前时间：18:33:50
Susp

Wiper & Washer - Tool - Wiper Arm Tool  <->  [ok] - 剩余数量：26 - 当前时间：18:34:17
Wiper & Washer - Wiper Blade - 15" Blade  <->  [ok] - 剩余数量：25 - 当前时间：18:34:17
Wheel - Tool - Wheel Weight Tool  <->  [ok] - 剩余数量：24 - 当前时间：18:34:17
Interior - Fluid / Chemical - Interior Cleaner  <->  [ok] - 剩余数量：23 - 当前时间：18:34:17
Wiper & Washer - Wiper Blade - 18" Blade  <->  [ok] - 剩余数量：22 - 当前时间：18:34:17
Wiper & Washer - Wiper Blade - 19" Blade  <->  [ok] - 剩余数量：21 - 当前时间：18:34:18
Wiper & Washer - Wiper Blade - 16" Blade  <->  [ok] - 剩余数量：20 - 当前时间：18:34:19
Wiper & Washer - Wiper Blade - 25" Blade  <->  [ok] - 剩余数量：19 - 当前时间：18:34:19
Wiper & Washer - Wiper Blade - 12" Blade  <->  [ok] - 剩余数量：18 - 当前时间：18:34:20
Wiper & Washer - Wiper Blade - 26" Blade  <->  [ok] - 剩余数量：17 - 当前时间：18:34:20
Wheel - Tool - Lug Stud Tool  <->  [ok] - 剩余数量：16 - 当前时间：18:34:20
Wiper & Washer - Wiper Blade - 17" Blade  <->  [ok] - 剩余数量：15 - 当前时间：18:34:21
Wiper & Washer - Wiper Blade - 28" Blade  <->  [ok] - 剩余数量：14 - 当前时间：18:34:21
Wi